In [358]:
import pandas as pd
# from pathlib import Path
# import eyed3
# import os
# from config import *
# import librosa
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline
# from PIL import Image
# import pathlib
# import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
from keras import models, layers

# import warnings
# warnings.filterwarnings('ignore')

# eyed3.log.setLevel("ERROR")

In [359]:
# Analysing the data in Pandas
data_raw = pd.read_csv(f'{genre_csvs_path}/all_genres_csv.csv')
data_raw.head()

,filename,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,genre
0,115384.mp3,0.360334,0.188969,2540.479255,2657.988433,5658.815782,0.079958,-37.924564,82.623207,-0.386473,...,4.505314,-0.521939,4.845899,-0.166509,3.191389,-5.220356,0.609465,-2.108234,3.528563,Chiptune
1,115390.mp3,0.378215,0.225411,2269.649055,2558.470365,5142.127240,0.045125,-5.867516,93.227295,-1.638364,...,0.458813,-5.935056,1.049750,-4.718104,1.299657,-4.143320,1.892623,-3.189257,2.489733,Chiptune
2,116882.mp3,0.295832,0.207426,2691.918208,2715.228839,5946.833547,0.093935,-28.662611,69.412605,-0.987070,...,-3.122667,-13.669146,-6.803550,-7.937631,3.512264,-2.820511,-1.227399,-11.329582,-1.202103,Chiptune
3,020673.mp3,0.308657,0.194924,1049.751106,921.605218,1869.794245,0.080677,-144.142548,211.651245,-61.986252,...,-0.187126,-0.222602,-0.353398,3.233407,3.270443,1.799441,5.244574,-1.119697,-5.840265,Chiptune
4,048295.mp3,0.478366,0.265123,2264.178956,2776.148175,5589.734312,0.061067,-83.822266,76.011131,20.975660,...,4.302501,-0.057032,3.677136,-2.304578,2.451861,-2.614267,0.899178,-4.417268,2.062210,Chiptune


In [360]:
data_raw.shape

(8334, 28)

In [361]:
# Dropping filename column
data_no_filenames = data_raw.drop(['filename'],axis=1)

In [362]:
data_no_filenames.shape
#data_no_filenames.head()

(8334, 27)

In [363]:
# Create three data sets, data_no_filenames, data_mfccs_only, and data_feat_mfccs
# CLASSIFICATION first features + MFCCs 1-13 only
# Remove mfcss over 13, retain genre column
mfcc = 'mfcc'
mfcc_list = [mfcc+str(x) for x in list(range(13+1 ,20+1))]
data_mfccs_plus = data_no_filenames.drop(mfcc_list, axis=1)
data_mfccs_plus.shape
print(data_mfccs_plus)

      chroma_stft       rms  spectral_centroid  spectral_bandwidth  \
0        0.360334  0.188969        2540.479255         2657.988433   
1        0.378215  0.225411        2269.649055         2558.470365   
2        0.295832  0.207426        2691.918208         2715.228839   
3        0.308657  0.194924        1049.751106          921.605218   
4        0.478366  0.265123        2264.178956         2776.148175   
...           ...       ...                ...                 ...   
8329     0.395774  0.120443        2058.111948         1851.790386   
8330     0.444157  0.318770        2117.818866         2310.499874   
8331     0.299300  0.108959        2102.942517         1539.198952   
8332     0.399944  0.102906        1626.045612         1678.233277   
8333     0.437711  0.231284        2131.379182         2197.591764   

          rolloff  zero_crossing_rate       mfcc1       mfcc2      mfcc3  \
0     5658.815782            0.079958  -37.924564   82.623207  -0.386473   
1     5

In [364]:
# Encoding the genres into numbers
genres_list = data_mfccs_plus.iloc[:, -1]
encoder = LabelEncoder()
genres_y = encoder.fit_transform(genres_list)
genres_y.shape
# print(genres_y)
# Just checking to see the original genres
#print(encoder.inverse_transform(genres_y))

(8334,)

In [365]:
# Get mfccs only
data_mfccs_only = data_mfccs_plus.iloc[:,data_mfccs_plus.columns.get_loc('mfcc1'):]
data_mfccs_only.shape

(8334, 14)

In [366]:
# Get first six features and mfcc2 only
mfcc2_drop = [mfcc+str(x) for x in list(range(2+1 ,13+1))]
data_feat_mfcc2 = data_mfccs_plus.drop(mfcc2_drop,axis=1)
data_feat_mfcc2 = data_feat_mfcc2.drop('mfcc1',axis=1)
data_feat_mfcc2.shape
#print(data_feat_mfcc2)

(8334, 8)

In [367]:
# Scaling the Feature Columns 
# a. All
scaler = StandardScaler()
features_X_a = scaler.fit_transform(np.array(data_no_filenames.iloc[:, :-1], dtype=float))
features_X_a.shape
#print(features_X_a)

(8334, 26)

In [368]:
# b. MFCCS 1-13 only
scaler = StandardScaler()
features_X_b = scaler.fit_transform(np.array(data_mfccs_only.iloc[:, :-1], dtype=float))
features_X_b.shape
print(features_X_b)

[[ 8.00164743e-01 -6.79844685e-01  5.39432988e-02 ...  3.62237406e-01
   3.49199914e-01  5.43141334e-01]
 [ 1.10827977e+00 -3.76665767e-01 -2.30739662e-04 ... -4.56610201e-01
  -3.64791492e-01 -4.97253381e-01]
 [ 8.89185633e-01 -1.05754581e+00  2.79532110e-02 ... -9.20478943e-01
  -9.96731409e-01 -1.98373640e+00]
 ...
 [-2.38475773e-01  9.69622147e-03 -3.23540036e+00 ... -3.54366378e-01
  -2.35372127e+00 -1.43822126e+00]
 [-2.51662491e-01  7.40185179e-01 -1.99117184e+00 ...  2.79560239e-01
   2.09361318e-01 -3.58570393e-01]
 [ 1.17072819e+00 -1.62979336e-01 -1.14285177e+00 ...  1.15058401e+00
   3.07143548e-01  2.89053491e-01]]


In [369]:
# c. First six features and mfcc2 only
scaler = StandardScaler()
features_X_c = scaler.fit_transform(np.array(data_feat_mfcc2.iloc[:, :-1], dtype=float))
features_X_c.shape
#print(features_X_c)

(8334, 7)

In [370]:
# CLASSIFICATION ALL FEATURES
# Dividing data into training and Testing set
# Test a
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(features_X_a, genres_y, test_size=0.2)
# Test b
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(features_X_b, genres_y, test_size=0.2)
# Test c
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(features_X_c, genres_y, test_size=0.2)

In [371]:
print(len(X_train_a), len(X_train_b), len(X_train_c))
print(len(X_test_a), len(X_test_b), len(X_test_c))
print(len(y_train_a), len(y_train_b), len(y_train_c))
print(len(y_test_a), len(y_test_b), len(y_test_c))
print(len(genres_y))

6667 6667 6667
1667 1667 1667
6667 6667 6667
1667 1667 1667
8334


In [372]:
# Classification with Keras
# Relu - Applies the rectified linear unit activation function.
# With default values, this returns the standard ReLU activation: max(x, 0), the element-wise maximum of 0 and the input tensor.
# https://keras.io/api/layers/activations/
# Softmax - The softmax function, also known as softargmax[1]:184 or normalized exponential function,[2]:198 is a generalization of the logistic 
# function to multiple dimensions. It is used in multinomial logistic regression and is often used as the last activation function of a neural 
# network to normalize the output of a network to a probability distribution over predicted output classes, based on Luce's choice axiom.
# https://en.wikipedia.org/wiki/Softmax_function

# Building our Networks
# A
modelA = models.Sequential()
modelA.add(layers.Dense(512, activation='relu', input_shape=(X_train_a.shape[1],)))
modelA.add(layers.Dense(256, activation='relu'))
modelA.add(layers.Dense(128, activation='relu'))
modelA.add(layers.Dense(64, activation='relu'))
modelA.add(layers.Dense(20, activation='softmax'))

# B
modelB = models.Sequential()
modelB.add(layers.Dense(512, activation='relu', input_shape=(X_train_b.shape[1],)))
modelB.add(layers.Dense(256, activation='relu'))
modelB.add(layers.Dense(128, activation='relu'))
modelB.add(layers.Dense(64, activation='relu'))
modelB.add(layers.Dense(20, activation='softmax'))

# C
modelC = models.Sequential()
modelC.add(layers.Dense(512, activation='relu', input_shape=(X_train_c.shape[1],)))
modelC.add(layers.Dense(256, activation='relu'))
modelC.add(layers.Dense(128, activation='relu'))
modelC.add(layers.Dense(64, activation='relu'))
modelC.add(layers.Dense(20, activation='softmax'))

In [373]:
# Config the model with losses and metrics
# Optimizer that implements the Adam algorithm - https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
# SparseCategoricalCrossentropy computes the crossentropy loss between the labels and predictions. Use this function
#   when there are two or more label classes. We expect labels to be provided as integers.
# Accuracy calculates how often predictions equal labels.  
modelA.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

modelB.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

modelC.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [374]:
# Set epochs
epochs = 20

In [375]:
# Train the model
# x_train = Input data, numpy array
# y_train = Target data, numpy array consistent with x
# epochs = integer. Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided.
#           The model is not trained for a number of iterations given by epochs, but merely until the epoch of index
#           epochs is reached.
# batch_size = Integer or None. Number of samples per gradient update. 
historyA = modelA.fit(X_train_a, y_train_a, epochs=epochs, batch_size=128)

Epoch 1/20
53/53 [==============================] - 1s 3ms/step - loss: 2.4856 - accuracy: 0.2959
Epoch 2/20
53/53 [==============================] - 0s 3ms/step - loss: 1.7513 - accuracy: 0.4623
Epoch 3/20
53/53 [==============================] - 0s 3ms/step - loss: 1.6250 - accuracy: 0.4982
Epoch 4/20
53/53 [==============================] - 0s 3ms/step - loss: 1.5022 - accuracy: 0.5331
Epoch 5/20
53/53 [==============================] - 0s 3ms/step - loss: 1.4571 - accuracy: 0.5368
Epoch 6/20
53/53 [==============================] - 0s 3ms/step - loss: 1.3571 - accuracy: 0.5604
Epoch 7/20
53/53 [==============================] - 0s 4ms/step - loss: 1.3083 - accuracy: 0.5809
Epoch 8/20
53/53 [==============================] - 0s 4ms/step - loss: 1.2288 - accuracy: 0.6078
Epoch 9/20
53/53 [==============================] - 0s 3ms/step - loss: 1.2196 - accuracy: 0.6146
Epoch 10/20
53/53 [==============================] - 0s 3ms/step - loss: 1.1522 - accuracy: 0.6279
Epoch 11/20
53/53 [

In [376]:
historyB = modelB.fit(X_train_b, y_train_b, epochs=epochs, batch_size=128)

Epoch 1/20
53/53 [==============================] - 1s 3ms/step - loss: 2.5739 - accuracy: 0.2777
Epoch 2/20
53/53 [==============================] - 0s 3ms/step - loss: 1.9357 - accuracy: 0.4101
Epoch 3/20
53/53 [==============================] - 0s 3ms/step - loss: 1.8800 - accuracy: 0.4211
Epoch 4/20
53/53 [==============================] - 0s 4ms/step - loss: 1.8260 - accuracy: 0.4309
Epoch 5/20
53/53 [==============================] - 0s 3ms/step - loss: 1.7527 - accuracy: 0.4468
Epoch 6/20
53/53 [==============================] - 0s 3ms/step - loss: 1.7010 - accuracy: 0.4629
Epoch 7/20
53/53 [==============================] - 0s 4ms/step - loss: 1.6723 - accuracy: 0.4778
Epoch 8/20
53/53 [==============================] - 0s 3ms/step - loss: 1.6562 - accuracy: 0.4750
Epoch 9/20
53/53 [==============================] - 0s 4ms/step - loss: 1.6086 - accuracy: 0.4952
Epoch 10/20
53/53 [==============================] - 0s 3ms/step - loss: 1.5757 - accuracy: 0.5017
Epoch 11/20
53/53 [

In [377]:
historyC = modelC.fit(X_train_c, y_train_c, epochs=epochs, batch_size=128)

Epoch 1/20
53/53 [==============================] - 1s 3ms/step - loss: 2.6714 - accuracy: 0.2706
Epoch 2/20
53/53 [==============================] - 0s 3ms/step - loss: 2.1739 - accuracy: 0.3683
Epoch 3/20
53/53 [==============================] - 0s 3ms/step - loss: 2.0440 - accuracy: 0.3900
Epoch 4/20
53/53 [==============================] - 0s 3ms/step - loss: 2.0489 - accuracy: 0.3698
Epoch 5/20
53/53 [==============================] - 0s 3ms/step - loss: 1.9965 - accuracy: 0.3850
Epoch 6/20
53/53 [==============================] - 0s 3ms/step - loss: 1.9853 - accuracy: 0.3942
Epoch 7/20
53/53 [==============================] - 0s 3ms/step - loss: 1.9638 - accuracy: 0.3964
Epoch 8/20
53/53 [==============================] - 0s 3ms/step - loss: 1.9216 - accuracy: 0.4119
Epoch 9/20
53/53 [==============================] - 0s 4ms/step - loss: 1.9205 - accuracy: 0.4105
Epoch 10/20
53/53 [==============================] - 0s 3ms/step - loss: 1.8820 - accuracy: 0.4204
Epoch 11/20
53/53 [

In [378]:
# Returns the loss value and metrics values for the model in test mode. Computation is done in batches.
# X_test = Input data
# y_test = Target data
# batch_size = Number of samples per batch of computation. If unspecified, will default to 32.
# test_loss, test_acc = model.evaluate(X_test, y_test)
testA = modelA.evaluate(X_test_a, y_test_a, verbose=0)
testB = modelB.evaluate(X_test_b, y_test_b, verbose=0)
testC = modelC.evaluate(X_test_c, y_test_c, verbose=0)
 
print('Test loss A: ', testA[0])
print('Test loss B: ', testB[0])
print('Test loss C: ', testC[0])

print('Test accuracy A: ', testA[1])
print('Test accuracy B: ', testB[1])
print('Test accuracy C: ', testC[1])

Test loss A:  1.797038197517395
Test loss B:  1.794525146484375
Test loss C:  1.888838529586792
Test accuracy A:  0.5134972929954529
Test accuracy B:  0.47150570154190063
Test accuracy C:  0.42111578583717346


In [379]:
# Validating approach
# Set apart 200 samples in training data to use as validation set
x_val_a = X_train_a[:200]
x_val_b = X_train_b[:200]
x_val_c = X_train_c[:200]
partial_x_train_a = X_train_a[200:]
partial_x_train_b = X_train_b[200:]
partial_x_train_c = X_train_c[200:]

y_val_a = y_train_a[:200]
y_val_b = y_train_b[:200]
y_val_c = y_train_c[:200]
partial_y_train_a = y_train_a[200:]
partial_y_train_b = y_train_b[200:]
partial_y_train_c = y_train_c[200:]

In [380]:
# Train network for epochs = 20
# Building our Networks
# A
modelA = models.Sequential()
modelA.add(layers.Dense(512, activation='relu', input_shape=(X_train_a.shape[1],)))
modelA.add(layers.Dense(256, activation='relu'))
modelA.add(layers.Dense(128, activation='relu'))
modelA.add(layers.Dense(64, activation='relu'))
modelA.add(layers.Dense(20, activation='softmax'))

# B
modelB = models.Sequential()
modelB.add(layers.Dense(512, activation='relu', input_shape=(X_train_b.shape[1],)))
modelB.add(layers.Dense(256, activation='relu'))
modelB.add(layers.Dense(128, activation='relu'))
modelB.add(layers.Dense(64, activation='relu'))
modelB.add(layers.Dense(20, activation='softmax'))

# C
modelC = models.Sequential()
modelC.add(layers.Dense(512, activation='relu', input_shape=(X_train_c.shape[1],)))
modelC.add(layers.Dense(256, activation='relu'))
modelC.add(layers.Dense(128, activation='relu'))
modelC.add(layers.Dense(64, activation='relu'))
modelC.add(layers.Dense(20, activation='softmax'))

modelA.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

modelB.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

modelC.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [381]:
historyA = modelA.fit(partial_x_train_a, partial_y_train_a, epochs=epochs, batch_size=512, validation_data=(x_val_a, y_val_a))
resultsA = modelA.evaluate(X_test_a, y_test_a)
print(resultsA)

NameError: name 'partial_y_train_a' is not defined

In [ ]:
historyB = modelB.fit(partial_x_train_b, partial_y_train_b, epochs=epochs, batch_size=512, validation_data=(x_val_b, y_val_b))
resultsB = modelB.evaluate(X_test_b, y_test_b)
print(resultsB)

In [ ]:
historyC = modelC.fit(partial_x_train_c, partial_y_train_c, epochs=epochs, batch_size=512, validation_data=(x_val_c, y_val_c))
resultsC = modelC.evaluate(X_test_c, y_test_c)
print(resultsC)